In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x394c4000 @  0x7f7a36b6d615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 22kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 10.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588511 sha256=b5419b5726be0d0aa89c4a22d45e684bfb10685c57a4bd00ce8b02c9d1ceed69
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 10.8

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-d4ykj22w
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-d4ykj22w
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=02f589775a139eb614b42b1db11b622aade75972525430efd200fa31c19f5238
  Stored in directory: /tmp/pip-ephem-wheel-cache-97dzbi08/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
device = torch.device("cuda:0")

In [ ]:
sentense = pd.read_excel('drive/My Drive/tagging.xlsx', sheet_name='문장f')

In [ ]:
sentense = sentense.replace({'[\(+\)]':' '}, regex = True)
sentense = sentense.replace({'\r':'','\n':'', '&gt':''}, regex = True)
sentense = sentense.replace({'[^A-Za-z0-9가-힣%. ]':''}, regex = True)
sentense = sentense.replace({' +':' '}, regex = True)

In [ ]:
sentense = sentense.drop('대표여부', axis = 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
result = le.fit_transform(sentense['태그'])
le_idx = dict(zip(list(le.classes_), le.transform(list(le.classes_))))
print(le_idx)

{'가설 설정': 0, '기술 정의': 1, '대상 데이터': 2, '데이터처리': 3, '문제 정의': 4, '성능효과': 5, '이론모형': 6, '제안 방법': 7, '후속연구': 8}


In [ ]:
sentense['태그'] = result

In [ ]:
sentense_train, sentense_test = train_test_split(sentense, test_size = 0.2)
sentense_train = sentense_train.reset_index().drop('index', axis = 1)
sentense_test = sentense_test.reset_index().drop('index', axis = 1)

In [ ]:
sentense_train.to_csv("drive/MyDrive/train_sen.csv", encoding='utf-8', index = False, sep='\t')
sentense_test.to_csv("drive/MyDrive/test_sen.csv", encoding='utf-8', index = False, sep='\t')

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
using cached model


In [ ]:
dataset_train = nlp.data.TSVDataset("drive/MyDrive/train_sen.csv", field_indices=[2,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("drive/MyDrive/test_sen.csv", field_indices=[2,1], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, bert,hidden_size = 768,num_classes=9,dr_rate=None,params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 2.175651788711548 train acc 0.15625
epoch 1 batch id 201 loss 1.350142478942871 train acc 0.2997512437810945
epoch 1 batch id 401 loss 1.3387243747711182 train acc 0.463606608478803
epoch 1 batch id 601 loss 0.9212745428085327 train acc 0.5365016638935108
epoch 1 batch id 801 loss 0.8280017375946045 train acc 0.5756476279650437
epoch 1 batch id 1001 loss 0.9061021208763123 train acc 0.6016483516483516
epoch 1 batch id 1201 loss 0.7857764959335327 train acc 0.6199261032472939
epoch 1 batch id 1401 loss 0.7094322443008423 train acc 0.6330968950749465

epoch 1 train acc 0.643360993514682


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.7400818639798489


epoch 2 batch id 1 loss 1.360571026802063 train acc 0.5
epoch 2 batch id 201 loss 0.5420540571212769 train acc 0.724657960199005
epoch 2 batch id 401 loss 1.2431529760360718 train acc 0.7323098503740648
epoch 2 batch id 601 loss 0.7762794494628906 train acc 0.7400686356073212
epoch 2 batch id 801 loss 0.6117644906044006 train acc 0.7457084893882646
epoch 2 batch id 1001 loss 0.6727102398872375 train acc 0.7525599400599401
epoch 2 batch id 1201 loss 0.5486047267913818 train acc 0.7574417152373022
epoch 2 batch id 1401 loss 0.5784656405448914 train acc 0.760728943611706

epoch 2 train acc 0.7643245586380832



epoch 2 test acc 0.7444112090680101


epoch 3 batch id 1 loss 0.9869948029518127 train acc 0.6875
epoch 3 batch id 201 loss 0.3071742355823517 train acc 0.7953980099502488
epoch 3 batch id 401 loss 0.9783675670623779 train acc 0.8012001246882793
epoch 3 batch id 601 loss 0.5400475859642029 train acc 0.8068843594009983
epoch 3 batch id 801 loss 0.4377322494983673 train acc 0.8121878901373284
epoch 3 batch id 1001 loss 0.49393001198768616 train acc 0.8175574425574426
epoch 3 batch id 1201 loss 0.5398910045623779 train acc 0.8207223147377186
epoch 3 batch id 1401 loss 0.3059217035770416 train acc 0.8226713062098501

epoch 3 train acc 0.8249358223743469



epoch 3 test acc 0.7461429471032746


epoch 4 batch id 1 loss 0.7589715719223022 train acc 0.6875
epoch 4 batch id 201 loss 0.2747800052165985 train acc 0.8519900497512438
epoch 4 batch id 401 loss 0.9488330483436584 train acc 0.8576995012468828
epoch 4 batch id 601 loss 0.34853753447532654 train acc 0.8620528286189684
epoch 4 batch id 801 loss 0.3185758590698242 train acc 0.864856429463171
epoch 4 batch id 1001 loss 0.5260942578315735 train acc 0.8686001498501499
epoch 4 batch id 1201 loss 0.3707166612148285 train acc 0.8715133222314738
epoch 4 batch id 1401 loss 0.27184829115867615 train acc 0.8724348679514632

epoch 4 train acc 0.8733308187713926



epoch 4 test acc 0.7474811083123426


In [ ]:
PATH = './model'

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')

In [ ]:
path = '/drive/MyDrive/'
torch.save(model, path+'model.pt')

FileNotFoundError: ignored